In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mkechinov/ecommerce-events-history-in-cosmetics-shop")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/ecommerce-events-history-in-cosmetics-shop


In [2]:
import os
print(os.listdir(path))

['2019-Nov.csv', '2020-Feb.csv', '2019-Oct.csv', '2019-Dec.csv', '2020-Jan.csv']


In [3]:
li = os.listdir(path)

In [4]:
li = [ '2019-Oct.csv','2019-Nov.csv','2019-Dec.csv', '2020-Jan.csv','2020-Feb.csv']

In [5]:
file_paths = []
for i in li:
  loc = path+ "/" +i
  file_paths.append(loc)
file_paths

['/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Oct.csv',
 '/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Nov.csv',
 '/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Dec.csv',
 '/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2020-Jan.csv',
 '/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2020-Feb.csv']

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [7]:
df_list = [pd.read_csv(file) for file in file_paths]

In [9]:
df_list[0]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
...,...,...,...,...,...,...,...,...,...
4102278,2019-10-31 23:59:25 UTC,remove_from_cart,5848062,1526733091857498510,NaN,NaN,9.21,429913900,2f0bff3c-252f-4fe6-afcd-5d8a6a92839a
4102279,2019-10-31 23:59:30 UTC,view,5885631,1783999064103190764,NaN,NaN,15.48,556138645,57ed222e-a54a-4907-9944-5a875c2d7f4f
4102280,2019-10-31 23:59:34 UTC,view,5784898,1487580009143992338,NaN,NaN,1.78,564441099,8cbf1746-2d46-4169-afb1-b5fa38ee1ba4
4102281,2019-10-31 23:59:36 UTC,view,5870838,1487580007675986893,NaN,milv,0.79,429913900,2f0bff3c-252f-4fe6-afcd-5d8a6a92839a


In [8]:
df = pd.concat(df_list, ignore_index = True)

In [9]:
df.shape

(20692840, 9)

In [10]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,NaN,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,NaN,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,NaN,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9


In [13]:
df.describe()

,product_id,category_id,price,user_id
count,2.069284e+07,2.069284e+07,2.069284e+07,2.069284e+07
mean,5.484297e+06,1.554230e+18,8.534735e+00,5.215527e+08
std,1.305716e+06,1.691038e+17,1.938142e+01,8.744312e+07
min,3.752000e+03,1.487580e+18,-7.937000e+01,4.654960e+05
25%,5.724650e+06,1.487580e+18,2.060000e+00,4.818306e+08
50%,5.810720e+06,1.487580e+18,4.050000e+00,5.531297e+08
75%,5.857864e+06,1.487580e+18,7.060000e+00,5.788573e+08
max,5.932595e+06,2.242903e+18,3.277800e+02,6.220902e+08


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20692840 entries, 0 to 20692839
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1.4+ GB


In [11]:
df.isna().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    20339246
brand             8757117
price                   0
user_id                 0
user_session         4598
dtype: int64

In [14]:
(df.isna().sum()/df.shape[0])*100

event_time        0.000000
event_type        0.000000
product_id        0.000000
category_id       0.000000
category_code    98.291225
brand            42.319551
price             0.000000
user_id           0.000000
user_session      0.022220
dtype: float64

In [17]:
df.columns

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session'],
      dtype='object')

In [12]:
import pandas as pd

# Assuming df is your concatenated DataFrame (~17.5M–20M rows)

# Step 1: Analyze product_id distribution
null_brand_rows = df[df['brand'].isnull()]
null_product_ids = null_brand_rows['product_id'].unique()
print(f"Unique product_ids in null brand rows: {len(null_product_ids)}")

Unique product_ids in null brand rows: 35726


In [14]:


# Step 2: Create product_id to brand mapping with frequency weighting
product_brand_map = (
    df[df['brand'].notnull()]
    .groupby(['product_id', 'brand'])
    .size()
    .reset_index(name='count')
    .sort_values(['product_id', 'count'], ascending=[True, False])
    .groupby('product_id')
    .first()
    ['brand']
    .to_dict()
)

In [15]:


# Step 3: Check mapping coverage
mapped_product_ids = list(product_brand_map.keys())
rows_covered = df[df['product_id'].isin(mapped_product_ids)].shape[0]
null_rows_mapped = null_brand_rows[null_brand_rows['product_id'].isin(mapped_product_ids)].shape[0]
print(f"Rows covered by product_ids with brands: {rows_covered}")
print(f"Null brand rows with mappable product_ids: {null_rows_mapped}")


Rows covered by product_ids with brands: 12034042
Null brand rows with mappable product_ids: 98319


In [16]:
# Step 4: Backup and impute
df['brand_original'] = df['brand']
df['brand'] = df['brand'].where(df['brand'].notnull(), df['product_id'].map(product_brand_map))

In [17]:
# Step 5: Validate imputation
initial_nulls = df['brand_original'].isnull().sum()
remaining_nulls = df['brand'].isnull().sum()
imputed_count = initial_nulls - remaining_nulls
print(f"Initial nulls in brand: {initial_nulls} ({initial_nulls/len(df)*100:.2f}%)")
print(f"Remaining nulls in brand: {remaining_nulls} ({remaining_nulls/len(df)*100:.2f}%)")
print(f"Rows imputed: {imputed_count} ({imputed_count/initial_nulls*100:.2f}%)")

Initial nulls in brand: 8757117 (42.32%)
Remaining nulls in brand: 8658798 (41.84%)
Rows imputed: 98319 (1.12%)


In [18]:
# Step 6: Check imputed rows
imputed_rows = df[df['brand_original'].isnull() & df['brand'].notnull()]
print(f"Top 5 product_ids imputed:\n", imputed_rows['product_id'].value_counts().head())

Top 5 product_ids imputed:
 product_id
5677463    2596
5677462    2324
5677465    2014
5676828    1553
5693593    1389
Name: count, dtype: int64


# since only 1.12% of Imputation is done and still 41.84% of brand is null so imputing the remaining null values with "unknown_brand"

In [19]:
df.isna().sum()

event_time               0
event_type               0
product_id               0
category_id              0
category_code     20339246
brand              8658798
price                    0
user_id                  0
user_session          4598
brand_original     8757117
dtype: int64

In [20]:
df['brand'] = df['brand'].fillna("unknown_brand")

In [21]:
df.isna().sum()

event_time               0
event_type               0
product_id               0
category_id              0
category_code     20339246
brand                    0
price                    0
user_id                  0
user_session          4598
brand_original     8757117
dtype: int64

# since category_code has 98% null percentage directly dropping it, and dropping the reference column brand_original

In [22]:
df.drop(columns = ['category_code','brand_original'], axis = 1, inplace =True)

In [23]:
df.head(2)

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885


In [24]:
df.isna().sum()

event_time         0
event_type         0
product_id         0
category_id        0
brand              0
price              0
user_id            0
user_session    4598
dtype: int64

In [25]:
df.shape

(20692840, 8)

# since user_session has 0.02% negligble amount hence dropping rows where user session null value is present

In [26]:
df = df.dropna(subset=['user_session'])

In [27]:
df.shape

(20688242, 8)

In [28]:
# df.dropna(how = 'any', axis = 0, inplace = True)

In [27]:
df.isna().sum()

event_time      0
event_type      0
product_id      0
category_id     0
brand           0
price           0
user_id         0
user_session    0
dtype: int64

In [28]:
df['event_type'].unique()

array(['cart', 'view', 'remove_from_cart', 'purchase'], dtype=object)

In [28]:
df['Purchase_Outcome'] = df['event_type'].apply(lambda x:1  if x=='purchase' else 0)

In [29]:
df.head()

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,Purchase_Outcome
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,0
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,0
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,0
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,0
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,0


In [30]:
df['Purchase_Outcome'].value_counts()

Purchase_Outcome
0    19401235
1     1287007
Name: count, dtype: int64

In [31]:
df[df['event_type'] == 'purchase']

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,Purchase_Outcome
202,2019-10-01 00:06:35 UTC,purchase,5619862,1487580006895846315,runail,5.32,474232307,445f2b74-5e4c-427e-b7fa-6e0a28b156fe,1
204,2019-10-01 00:06:35 UTC,purchase,5619866,1487580006895846315,runail,2.86,474232307,445f2b74-5e4c-427e-b7fa-6e0a28b156fe,1
205,2019-10-01 00:06:35 UTC,purchase,5696184,1487580005134238553,runail,2.38,474232307,445f2b74-5e4c-427e-b7fa-6e0a28b156fe,1
206,2019-10-01 00:06:35 UTC,purchase,5706784,1487580005092295511,runail,4.44,474232307,445f2b74-5e4c-427e-b7fa-6e0a28b156fe,1
207,2019-10-01 00:06:35 UTC,purchase,5723503,1487580005134238553,runail,2.62,474232307,445f2b74-5e4c-427e-b7fa-6e0a28b156fe,1
...,...,...,...,...,...,...,...,...,...
20692507,2020-02-29 23:46:01 UTC,purchase,5896420,1487580006300255120,jessnail,49.21,620865261,64389dba-26c4-f328-6972-166a48654eaa,1
20692721,2020-02-29 23:54:44 UTC,purchase,5692863,1487580007910867929,yoko,7.83,548866558,199869f5-3a22-4a47-8678-7c343d09bb8e,1
20692722,2020-02-29 23:54:44 UTC,purchase,5802432,1487580009286598681,unknown_brand,0.32,548866558,199869f5-3a22-4a47-8678-7c343d09bb8e,1
20692723,2020-02-29 23:54:44 UTC,purchase,5896788,1783999068909863670,staleks,4.75,548866558,199869f5-3a22-4a47-8678-7c343d09bb8e,1


In [32]:
df['event_type'].unique()

array(['cart', 'view', 'remove_from_cart', 'purchase'], dtype=object)

In [31]:
funnel_stages = ['view', 'cart', 'purchase']
stage_labels = ['Visit page', 'Add to cart', 'Purchase complete']

In [32]:
df['event_type'] = df['event_type'].str.strip()

In [33]:
funnel_counts = [df[df['event_type'] == stage].shape[0] for stage in funnel_stages]
funnel_counts

[9657790, 5764557, 1287007]

In [34]:
df['event_type'].value_counts()

event_type
view                9657790
cart                5764557
remove_from_cart    3978888
purchase            1287007
Name: count, dtype: int64

In [36]:
conversion_rates1 = [100] + [(funnel_counts[i] / funnel_counts[i-1]) * 100 for i in range(1, len(funnel_counts))]
conversion_rates1

[100, 59.68815847103737, 22.326208241153655]

In [37]:
conversion_rates = []
for ind, value in enumerate(funnel_counts):
  if ind == 0:
    conversion_rates.append(100)
  else:
    co_rate = (value/funnel_counts[ind-1]) * 100
    conversion_rates.append(co_rate)
conversion_rates

[100, 59.68815847103737, 22.326208241153655]

In [38]:
dropoff_rates = [100 - rate for rate in conversion_rates]
dropoff_rates

[0, 40.31184152896263, 77.67379175884635]

In [39]:
remove_from_cart_count = df[df['event_type'] == 'remove_from_cart'].shape[0]
remove_from_cart_rate = (remove_from_cart_count/funnel_counts[1]) *100 if funnel_counts[1]>0 else 0 

In [40]:
import plotly.graph_objects as go

In [41]:
print("Funnel Analysis:")
for stage, count, conv, drop in zip(stage_labels, funnel_counts, conversion_rates, dropoff_rates):
    print(f"{stage}: {count} users, Conversion Rate: {conv:.2f}%, Drop-Off Rate: {drop:.2f}%")
print(f"Remove from cart: {remove_from_cart_count} events, Proportion of cart events: {remove_from_cart_rate:.2f}%")

Funnel Analysis:
Visit page: 9657790 users, Conversion Rate: 100.00%, Drop-Off Rate: 0.00%
Add to cart: 5764557 users, Conversion Rate: 59.69%, Drop-Off Rate: 40.31%
Purchase complete: 1287007 users, Conversion Rate: 22.33%, Drop-Off Rate: 77.67%
Remove from cart: 3978888 events, Proportion of cart events: 69.02%


In [42]:
fig = go.Figure(go.Funnel(
    y= stage_labels,
    x=funnel_counts,
    textinfo="value+percent initial",
    marker={"color": ["#FF6384", "#36A2EB", "#FFCE56", "#4BC0C0", "#9966FF"]}

))
fig.update_layout(title="Ecommerce Purchase Funnel", showlegend=False)
fig.show()

In [43]:
df.head()

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,Purchase_Outcome
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,0
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,0
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,0
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,0
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,0


In [44]:
df.dtypes

event_time           object
event_type           object
product_id            int64
category_id           int64
brand                object
price               float64
user_id               int64
user_session         object
Purchase_Outcome      int64
dtype: object

In [45]:
df['price'].describe()

count    2.068824e+07
mean     8.534892e+00
std      1.938206e+01
min     -7.937000e+01
25%      2.060000e+00
50%      4.050000e+00
75%      7.030000e+00
max      3.277800e+02
Name: price, dtype: float64

# Since there is no positive prices for these product_id's, dropping these rows

In [46]:
df = df[df['price']>0]

In [47]:
df['event_time'] = pd.to_datetime(df['event_time'])
df.head(2)

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,Purchase_Outcome
0,2019-10-01 00:00:00+00:00,cart,5773203,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,0
1,2019-10-01 00:00:03+00:00,cart,5773353,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,0


In [69]:
purchase_data = df[df['Purchase_Outcome']==1].groupby('user_id').agg({
    'price': ['sum','mean']
}).reset_index()
purchase_data.columns = ['user_id','monetary','avg_purchase_price']
purchase_data.head(2)

,user_id,monetary,avg_purchase_price
0,9794320,12.68,3.170
1,10079204,25.81,12.905


In [70]:
user_agg = df.groupby('user_id').agg({
    'event_type': lambda x: x.value_counts().to_dict(),
    'price': ['sum', 'mean'],
    'user_session': 'nunique',
    'event_time': ['min','max'],
    'Purchase_Outcome': 'sum',
    'category_id': 'nunique'
}).reset_index()

In [71]:
user_agg.head(2)

user_id   event_type  price        user_session                event_time  \
               <lambda>    sum   mean      nunique                       min   
0   465496  {'view': 3}  30.00  10.00            2 2020-01-14 16:15:21+00:00   
1  1120748  {'view': 1}   4.05   4.05            1 2019-11-04 15:28:57+00:00   

                            Purchase_Outcome category_id  
                        max              sum     nunique  
0 2020-01-14 16:27:31+00:00                0           1  
1 2019-11-04 15:28:57+00:00                0           1

In [72]:
user_agg.columns = ['user_id', 'event_counts', 'total_interaction_price', 'avg_interaction_price',
                    'session_count',' first_event','last_event','purchase_count','category_diversity']
user_agg.head(2)

,user_id,event_counts,total_interaction_price,avg_interaction_price,session_count,first_event,last_event,purchase_count,category_diversity
0,465496,{'view': 3},30.00,10.00,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1
1,1120748,{'view': 1},4.05,4.05,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1


In [73]:
user_agg = pd.merge(user_agg,purchase_data, on = 'user_id', how = 'left')

In [54]:
user_agg.head(2)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in less

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in less

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning:

invalid value encountered in greater



,user_id,event_counts,total_interaction_price,avg_interaction_price,session_count,first_event,last_event,purchase_count,category_diversity,monetary,avg_purchase_price
0,465496,{'view': 3},30.00,10.00,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1,NaN,NaN
1,1120748,{'view': 1},4.05,4.05,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1,NaN,NaN


In [55]:
user_agg[user_agg['monetary'].isna()]['event_counts'].apply(lambda x: x.get("purchase",0)).value_counts()

event_counts
0    1527252
Name: count, dtype: int64

In [56]:
user_agg[user_agg['avg_purchase_price'].isna()]['event_counts'].apply(lambda x: x.get("purchase",0)).value_counts()

event_counts
0    1527252
Name: count, dtype: int64

In [74]:
user_agg['monetary'] = user_agg['monetary'].fillna(0)
user_agg['avg_purchase_price'] = user_agg['avg_purchase_price'].fillna(0)

In [75]:
user_agg.isna().sum()

user_id                    0
event_counts               0
total_interaction_price    0
avg_interaction_price      0
session_count              0
 first_event               0
last_event                 0
purchase_count             0
category_diversity         0
monetary                   0
avg_purchase_price         0
dtype: int64

In [76]:
reference_date = df['event_time'].max()
reference_date

Timestamp('2020-02-29 23:59:59+0000', tz='UTC')

In [77]:
# Feature Engeneering

## RFM Features
reference_date = df['event_time'].max()
user_agg['recency'] =( reference_date-user_agg['last_event']).dt.days
user_agg['frequency'] = user_agg['session_count']

## Behavioral Features
user_agg['view_count'] = user_agg['event_counts'].apply(lambda x: x.get('view',0))
user_agg['remove_count'] = user_agg['event_counts'].apply(lambda x: x.get('remove_from_cart',0))
user_agg['cart_count'] = user_agg['event_counts'].apply(lambda x: x.get('cart',0))
user_agg['purchase_event_count'] = user_agg['event_counts'].apply(lambda x: x.get('purchase',0))
user_agg['total_events']  = user_agg['view_count'] + user_agg['remove_count'] + user_agg['cart_count'] + user_agg['purchase_event_count']
user_agg['conversion_rate'] = user_agg['purchase_count']/user_agg['total_events']
user_agg['cart_abandonment_rate'] = user_agg['remove_count']/(user_agg['cart_count']+1e-6)


In [78]:
## Session based aggregation

session_agg = df.groupby(['user_id','user_session']).agg({
    'event_time': ['min','max'],
    'event_type':'count'
}).reset_index()


session_agg.head(2)

user_id                          user_session                event_time  \
                                                                      min   
0  465496  8ad668e6-9567-4da9-b43b-250f12ecab5b 2020-01-14 16:15:21+00:00   
1  465496  d4b9b57f-237c-4427-80e1-b0073b0b0b6c 2020-01-14 16:25:35+00:00   

                            event_type  
                        max      count  
0 2020-01-14 16:15:21+00:00          1  
1 2020-01-14 16:27:31+00:00          2

In [79]:
session_agg.columns

MultiIndex([(     'user_id',      ''),
            ('user_session',      ''),
            (  'event_time',   'min'),
            (  'event_time',   'max'),
            (  'event_type', 'count')],
           )

In [80]:
session_agg['duration'] = (session_agg['event_time']['max']-session_agg['event_time']['min']).dt.seconds
session_agg.head(2)

user_id                          user_session                event_time  \
                                                                      min   
0  465496  8ad668e6-9567-4da9-b43b-250f12ecab5b 2020-01-14 16:15:21+00:00   
1  465496  d4b9b57f-237c-4427-80e1-b0073b0b0b6c 2020-01-14 16:25:35+00:00   

                            event_type duration  
                        max      count           
0 2020-01-14 16:15:21+00:00          1        0  
1 2020-01-14 16:27:31+00:00          2      116

In [81]:
session_agg.columns

MultiIndex([(     'user_id',      ''),
            ('user_session',      ''),
            (  'event_time',   'min'),
            (  'event_time',   'max'),
            (  'event_type', 'count'),
            (    'duration',      '')],
           )

In [82]:
session_agg.columns = ['user_id', 'user_session', 'event_time_min', 'event_time_max', 'event_type', 'duration']
session_agg.head(2)

,user_id,user_session,event_time_min,event_time_max,event_type,duration
0,465496,8ad668e6-9567-4da9-b43b-250f12ecab5b,2020-01-14 16:15:21+00:00,2020-01-14 16:15:21+00:00,1,0
1,465496,d4b9b57f-237c-4427-80e1-b0073b0b0b6c,2020-01-14 16:25:35+00:00,2020-01-14 16:27:31+00:00,2,116


In [83]:
session_summary = session_agg.groupby('user_id').agg({
    'duration': 'mean',
    'event_type': 'mean'
}).reset_index()

session_summary.columns = ['user_id','avg_session_duration','avg_events_per_session']
session_summary.head(2)

,user_id,avg_session_duration,avg_events_per_session
0,465496,58.0,1.5
1,1120748,0.0,1.0


In [84]:
## Merge session features
user_agg = pd.merge(user_agg, session_summary, on='user_id', how = 'left')
user_agg.head(2)

,user_id,event_counts,total_interaction_price,avg_interaction_price,session_count,first_event,last_event,purchase_count,category_diversity,monetary,...,frequency,view_count,remove_count,cart_count,purchase_event_count,total_events,conversion_rate,cart_abandonment_rate,avg_session_duration,avg_events_per_session
0,465496,{'view': 3},30.00,10.00,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1,0.0,...,2,3,0,0,0,3,0.0,0.0,58.0,1.5
1,1120748,{'view': 1},4.05,4.05,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1,0.0,...,1,1,0,0,0,1,0.0,0.0,0.0,1.0


In [85]:
user_hour = df.groupby('user_id')['event_time'].apply(lambda x: x.dt.hour.mode().iloc[0]).reset_index()
user_hour.columns = ['user_id', 'most_common_hour']
user_hour.head(2)

,user_id,most_common_hour
0,465496,16
1,1120748,15


In [86]:
user_agg.head(2)

,user_id,event_counts,total_interaction_price,avg_interaction_price,session_count,first_event,last_event,purchase_count,category_diversity,monetary,...,frequency,view_count,remove_count,cart_count,purchase_event_count,total_events,conversion_rate,cart_abandonment_rate,avg_session_duration,avg_events_per_session
0,465496,{'view': 3},30.00,10.00,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1,0.0,...,2,3,0,0,0,3,0.0,0.0,58.0,1.5
1,1120748,{'view': 1},4.05,4.05,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1,0.0,...,1,1,0,0,0,1,0.0,0.0,0.0,1.0


In [87]:
user_agg = pd.merge(user_agg, user_hour, on = 'user_id', how='left')
user_agg.head(2)

,user_id,event_counts,total_interaction_price,avg_interaction_price,session_count,first_event,last_event,purchase_count,category_diversity,monetary,...,view_count,remove_count,cart_count,purchase_event_count,total_events,conversion_rate,cart_abandonment_rate,avg_session_duration,avg_events_per_session,most_common_hour
0,465496,{'view': 3},30.00,10.00,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1,0.0,...,3,0,0,0,3,0.0,0.0,58.0,1.5,16
1,1120748,{'view': 1},4.05,4.05,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1,0.0,...,1,0,0,0,1,0.0,0.0,0.0,1.0,15


In [88]:
df.isna().sum()

event_time          0
event_type          0
product_id          0
category_id         0
brand               0
price               0
user_id             0
user_session        0
Purchase_Outcome    0
dtype: int64

In [69]:
df['category_id'].nunique()

523

In [73]:
df['event_type'].value_counts()

event_type
view                9627298
cart                5715188
remove_from_cart    3954589
purchase            1286880
Name: count, dtype: int64

In [76]:
df['category_id'].value_counts()

category_id
1487580007675986893    1045608
1487580005595612013     786934
1487580005092295511     769070
1487580005671109489     677263
1487580006317032337     643394
                        ...   
1487580007155893175          2
1487580009890578479          1
1487580011056595041          1
1487580009706029097          1
1487580010897211483          1
Name: count, Length: 523, dtype: int64

In [75]:
df[df['event_type'] == 'purchase']['category_id'].value_counts()

category_id
1487580007675986893    80137
1487580006317032337    50555
1487580005092295511    44870
1487580005595612013    44334
1487580005671109489    43451
                       ...  
1487580005318787937        1
1487580008951054345        1
2166295400451933025        1
1487580013564788937        1
1842735764450837316        1
Name: count, Length: 468, dtype: int64

In [89]:
category_freq = df['category_id'].value_counts().to_dict()

df['category_freq'] = df['category_id'].map(category_freq)
df.head(1)

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,Purchase_Outcome,category_freq
0,2019-10-01 00:00:00+00:00,cart,5773203,1487580005134238553,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,0,394389


In [90]:
avg_cat_freq = df.groupby('user_id')['category_freq'].mean().reset_index()
avg_cat_freq.columns = ['user_id','avg_category_freq' ]
avg_cat_freq.head()

,user_id,avg_category_freq
0,465496,39226.000000
1,1120748,31120.000000
2,1180452,119609.000000
3,1458813,71265.448276
4,2038666,28437.000000


In [91]:
user_agg = pd.merge(user_agg, avg_cat_freq, on ='user_id',how='left')
user_agg.head(2)

,user_id,event_counts,total_interaction_price,avg_interaction_price,session_count,first_event,last_event,purchase_count,category_diversity,monetary,...,remove_count,cart_count,purchase_event_count,total_events,conversion_rate,cart_abandonment_rate,avg_session_duration,avg_events_per_session,most_common_hour,avg_category_freq
0,465496,{'view': 3},30.00,10.00,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1,0.0,...,0,0,0,3,0.0,0.0,58.0,1.5,16,39226.0
1,1120748,{'view': 1},4.05,4.05,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1,0.0,...,0,0,0,1,0.0,0.0,0.0,1.0,15,31120.0


In [92]:
## Price Sensitivity features
view_prices = df[df['event_type']=='view'].groupby('user_id')['price'].mean().reset_index()
view_prices.columns = ['user_id','avg_view_price']
view_prices.head(2)

,user_id,avg_view_price
0,465496,10.00
1,1120748,4.05


In [93]:
user_agg = pd.merge(user_agg,view_prices,on='user_id',how ='left')
user_agg.head()

,user_id,event_counts,total_interaction_price,avg_interaction_price,session_count,first_event,last_event,purchase_count,category_diversity,monetary,...,cart_count,purchase_event_count,total_events,conversion_rate,cart_abandonment_rate,avg_session_duration,avg_events_per_session,most_common_hour,avg_category_freq,avg_view_price
0,465496,{'view': 3},30.00,10.000000,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1,0.0,...,0,0,3,0.0,0.0,58.00,1.50,16,39226.000000,10.000000
1,1120748,{'view': 1},4.05,4.050000,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1,0.0,...,0,0,1,0.0,0.0,0.00,1.00,15,31120.000000,4.050000
2,1180452,{'view': 1},25.40,25.400000,1,2019-12-28 14:32:56+00:00,2019-12-28 14:32:56+00:00,0,1,0.0,...,0,0,1,0.0,0.0,0.00,1.00,14,119609.000000,25.400000
3,1458813,{'view': 29},159.36,5.495172,4,2019-11-19 09:56:09+00:00,2019-11-21 06:02:40+00:00,0,5,0.0,...,0,0,29,0.0,0.0,947.25,7.25,12,71265.448276,5.495172
4,2038666,{'view': 1},25.38,25.380000,1,2020-02-16 13:19:37+00:00,2020-02-16 13:19:37+00:00,0,1,0.0,...,0,0,1,0.0,0.0,0.00,1.00,13,28437.000000,25.380000


In [94]:
user_agg.columns

Index(['user_id', 'event_counts', 'total_interaction_price',
       'avg_interaction_price', 'session_count', ' first_event', 'last_event',
       'purchase_count', 'category_diversity', 'monetary',
       'avg_purchase_price', 'recency', 'frequency', 'view_count',
       'remove_count', 'cart_count', 'purchase_event_count', 'total_events',
       'conversion_rate', 'cart_abandonment_rate', 'avg_session_duration',
       'avg_events_per_session', 'most_common_hour', 'avg_category_freq',
       'avg_view_price'],
      dtype='object')

In [95]:
user_agg.isna().sum()

user_id                        0
event_counts                   0
total_interaction_price        0
avg_interaction_price          0
session_count                  0
 first_event                   0
last_event                     0
purchase_count                 0
category_diversity             0
monetary                       0
avg_purchase_price             0
recency                        0
frequency                      0
view_count                     0
remove_count                   0
cart_count                     0
purchase_event_count           0
total_events                   0
conversion_rate                0
cart_abandonment_rate          0
avg_session_duration           0
avg_events_per_session         0
most_common_hour               0
avg_category_freq              0
avg_view_price             41411
dtype: int64

# so based on this avg_view_price column is null for records which has no "view" as the event type. hence imputing it with value 0

In [96]:
user_agg.columns

Index(['user_id', 'event_counts', 'total_interaction_price',
       'avg_interaction_price', 'session_count', ' first_event', 'last_event',
       'purchase_count', 'category_diversity', 'monetary',
       'avg_purchase_price', 'recency', 'frequency', 'view_count',
       'remove_count', 'cart_count', 'purchase_event_count', 'total_events',
       'conversion_rate', 'cart_abandonment_rate', 'avg_session_duration',
       'avg_events_per_session', 'most_common_hour', 'avg_category_freq',
       'avg_view_price'],
      dtype='object')

In [97]:
user_agg['avg_view_price'] = user_agg['avg_view_price'].fillna(0)

In [98]:
pd.set_option('display.max_columns', None)

In [99]:
user_agg['price_ratio'] = user_agg['avg_purchase_price'] / (user_agg['avg_view_price']+ 1e-6)

In [100]:
user_agg.isna().sum()

user_id                    0
event_counts               0
total_interaction_price    0
avg_interaction_price      0
session_count              0
 first_event               0
last_event                 0
purchase_count             0
category_diversity         0
monetary                   0
avg_purchase_price         0
recency                    0
frequency                  0
view_count                 0
remove_count               0
cart_count                 0
purchase_event_count       0
total_events               0
conversion_rate            0
cart_abandonment_rate      0
avg_session_duration       0
avg_events_per_session     0
most_common_hour           0
avg_category_freq          0
avg_view_price             0
price_ratio                0
dtype: int64

In [101]:
user_agg.columns

Index(['user_id', 'event_counts', 'total_interaction_price',
       'avg_interaction_price', 'session_count', ' first_event', 'last_event',
       'purchase_count', 'category_diversity', 'monetary',
       'avg_purchase_price', 'recency', 'frequency', 'view_count',
       'remove_count', 'cart_count', 'purchase_event_count', 'total_events',
       'conversion_rate', 'cart_abandonment_rate', 'avg_session_duration',
       'avg_events_per_session', 'most_common_hour', 'avg_category_freq',
       'avg_view_price', 'price_ratio'],
      dtype='object')

In [102]:
user_agg.head(2)

,user_id,event_counts,total_interaction_price,avg_interaction_price,session_count,first_event,last_event,purchase_count,category_diversity,monetary,avg_purchase_price,recency,frequency,view_count,remove_count,cart_count,purchase_event_count,total_events,conversion_rate,cart_abandonment_rate,avg_session_duration,avg_events_per_session,most_common_hour,avg_category_freq,avg_view_price,price_ratio
0,465496,{'view': 3},30.00,10.00,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1,0.0,0.0,46,2,3,0,0,0,3,0.0,0.0,58.0,1.5,16,39226.0,10.00,0.0
1,1120748,{'view': 1},4.05,4.05,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1,0.0,0.0,117,1,1,0,0,0,1,0.0,0.0,0.0,1.0,15,31120.0,4.05,0.0


In [103]:
user_agg.columns

Index(['user_id', 'event_counts', 'total_interaction_price',
       'avg_interaction_price', 'session_count', ' first_event', 'last_event',
       'purchase_count', 'category_diversity', 'monetary',
       'avg_purchase_price', 'recency', 'frequency', 'view_count',
       'remove_count', 'cart_count', 'purchase_event_count', 'total_events',
       'conversion_rate', 'cart_abandonment_rate', 'avg_session_duration',
       'avg_events_per_session', 'most_common_hour', 'avg_category_freq',
       'avg_view_price', 'price_ratio'],
      dtype='object')

# Now removing the unwanted columns.

In [104]:
# user_agg_final = user_agg.drop(columns = ['user_id','event_counts',' first_event','last_event'] , axis = 0)
user_agg_final = user_agg.drop(columns = ['event_counts',' first_event','last_event'] , axis = 0)

user_agg_final.head(2)

,user_id,total_interaction_price,avg_interaction_price,session_count,purchase_count,category_diversity,monetary,avg_purchase_price,recency,frequency,view_count,remove_count,cart_count,purchase_event_count,total_events,conversion_rate,cart_abandonment_rate,avg_session_duration,avg_events_per_session,most_common_hour,avg_category_freq,avg_view_price,price_ratio
0,465496,30.00,10.00,2,0,1,0.0,0.0,46,2,3,0,0,0,3,0.0,0.0,58.0,1.5,16,39226.0,10.00,0.0
1,1120748,4.05,4.05,1,0,1,0.0,0.0,117,1,1,0,0,0,1,0.0,0.0,0.0,1.0,15,31120.0,4.05,0.0


In [105]:
user_agg_final.shape

(1637770, 23)

# Checking vif using all numerical sensible variables

In [113]:
features = ['total_spend', 'avg_price', 'session_count',
        'purchase_count', 'category_diversity',
       'recency', 'frequency', 'monetary', 'view_count', 'remove_count',
       'cart_count','purchase_event_count', 'total_events', 'conversion_rate',
       'cart_abandonment_rate', 'avg_session_duration',
       'avg_events_per_session', 'most_common_hour', 'avg_category_freq',
       'avg_view_price', 'price_ratio']

In [ ]:
mat = user_agg[features].corr()
mask = np.triu(mat)
plt.figure(figsize=(18,15))
sns.heatmap(mat, cmap='Blues', annot=True,mask=mask)
plt.show()

In [115]:
X= user_agg[features]

In [ ]:
np.isinf(X).values.sum()

In [116]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_df= pd.DataFrame()
vif_df['features'] = X.columns
vif_df['VIF'] = [variance_inflation_factor(X.values,i) for i in range(len(X.columns))]
vif_df

/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning:

divide by zero encountered in scalar divide



,features,VIF
0,total_spend,inf
1,avg_price,80.793866
2,session_count,inf
3,purchase_count,inf
4,category_diversity,3.550452
5,recency,2.576960
6,frequency,inf
7,monetary,inf
8,view_count,inf
9,remove_count,inf


# Checking Vif using version1 columns

In [117]:
version1 = [
    'recency', 'frequency', 'monetary', 'conversion_rate',
    'cart_abandonment_rate', 'avg_session_duration', 'category_diversity'
]

In [118]:
X1= user_agg[version1]

In [119]:

vif_df1= pd.DataFrame()
vif_df1['features'] = X1.columns
vif_df1['VIF'] = [variance_inflation_factor(X1.values,i) for i in range(len(X1.columns))]
vif_df1

,features,VIF
0,recency,1.108701
1,frequency,1.363301
2,monetary,2.139575
3,conversion_rate,1.134035
4,cart_abandonment_rate,1.001028
5,avg_session_duration,1.043033
6,category_diversity,2.193036


# Checking for Vif using version 2 columns

In [120]:
version2 = [
    'recency', 'frequency', 'monetary', 'conversion_rate',
    'cart_abandonment_rate', 'avg_session_duration', 'category_diversity',
    'avg_category_freq', 'price_ratio'
]

In [121]:
X2= user_agg[version2]

In [122]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_df2= pd.DataFrame()
vif_df2['features'] = X2.columns
vif_df2['VIF'] = [variance_inflation_factor(X2.values,i) for i in range(len(X2.columns))]
vif_df2

,features,VIF
0,recency,1.586491
1,frequency,1.363751
2,monetary,2.143729
3,conversion_rate,1.139871
4,cart_abandonment_rate,1.001157
5,avg_session_duration,1.045222
6,category_diversity,2.264244
7,avg_category_freq,1.650818
8,price_ratio,1.009282


# Checking for vif using own version3

In [123]:
version3 = [
         'category_diversity',
       'recency', 'frequency', 'monetary', 'total_events', 'conversion_rate',
       'cart_abandonment_rate', 'avg_session_duration',
       'avg_events_per_session', 'most_common_hour', 'avg_category_freq',
       'avg_view_price', 'price_ratio']

In [124]:
X3= user_agg[version3]

In [125]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_df3= pd.DataFrame()
vif_df3['features'] = X3.columns
vif_df3['VIF'] = [variance_inflation_factor(X3.values,i) for i in range(len(X3.columns))]
vif_df3

,features,VIF
0,category_diversity,3.203232
1,recency,2.573331
2,frequency,1.586142
3,monetary,3.030649
4,total_events,4.327605
5,conversion_rate,1.154014
6,cart_abandonment_rate,1.002212
7,avg_session_duration,1.085222
8,avg_events_per_session,1.333216
9,most_common_hour,3.163793


In [126]:
version4 = [
         'category_diversity',
       'recency', 'frequency', 'monetary', 'conversion_rate',
       'cart_abandonment_rate', 'avg_session_duration',
       'avg_events_per_session', 'most_common_hour', 'avg_category_freq',
       'avg_view_price', 'price_ratio']
X4= user_agg[version4]
vif_df4= pd.DataFrame()
vif_df4['features'] = X4.columns
vif_df4['VIF'] = [variance_inflation_factor(X4.values,i) for i in range(len(X4.columns))]
vif_df4

,features,VIF
0,category_diversity,2.475347
1,recency,2.570935
2,frequency,1.402467
3,monetary,2.278848
4,conversion_rate,1.151070
5,cart_abandonment_rate,1.002105
6,avg_session_duration,1.084064
7,avg_events_per_session,1.287676
8,most_common_hour,3.147583
9,avg_category_freq,1.856656


In [127]:
version5 = [
         'category_diversity',
       'recency', 'frequency', 'monetary', 'conversion_rate',
       'cart_abandonment_rate', 'avg_session_duration',
       'avg_events_per_session', 'avg_category_freq',
       'avg_view_price', 'price_ratio']
X5= user_agg[version5]
vif_df5= pd.DataFrame()
vif_df5['features'] = X5.columns
vif_df5['VIF'] = [variance_inflation_factor(X5.values,i) for i in range(len(X5.columns))]
vif_df5

,features,VIF
0,category_diversity,2.378114
1,recency,1.750441
2,frequency,1.401622
3,monetary,2.262807
4,conversion_rate,1.150172
5,cart_abandonment_rate,1.002094
6,avg_session_duration,1.083942
7,avg_events_per_session,1.287201
8,avg_category_freq,1.668712
9,avg_view_price,1.269897


In [69]:
version6 = [

       'recency', 'frequency', 'monetary', 'conversion_rate',
       'cart_abandonment_rate', 'avg_session_duration',
       'category_diversity', 'avg_category_freq',
        'price_ratio']
X6= user_agg[version6]
vif_df6= pd.DataFrame()
vif_df6['features'] = X6.columns
vif_df6['VIF'] = [variance_inflation_factor(X6.values,i) for i in range(len(X6.columns))]
vif_df6

,features,VIF
0,recency,1.586491
1,frequency,1.363751
2,monetary,2.143729
3,conversion_rate,1.139871
4,cart_abandonment_rate,1.001157
5,avg_session_duration,1.045222
6,category_diversity,2.264244
7,avg_category_freq,1.650818
8,price_ratio,1.009282


In [70]:
user_agg[version6].isna().sum()

recency                  0
frequency                0
monetary                 0
conversion_rate          0
cart_abandonment_rate    0
avg_session_duration     0
category_diversity       0
avg_category_freq        0
price_ratio              0
dtype: int64

In [71]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [72]:
X = user_agg[version6]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 42)
user_agg['cluster'] = kmeans.fit_predict(X_scaled)

/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [73]:
user_agg.head()

,user_id,event_counts,total_spend,avg_price,session_count,first_event,last_event,purchase_count,category_diversity,recency,frequency,monetary,view_count,remove_count,cart_count,purchase_event_count,total_events,conversion_rate,cart_abandonment_rate,avg_session_duration,avg_events_per_session,most_common_hour,avg_category_freq,avg_view_price,price_ratio,cluster
0,465496,{'view': 3},30.00,10.000000,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1,46,2,30.00,3,0,0,0,3,0.0,0.0,58.00,1.50,16,39226.000000,10.000000,1.0,1
1,1120748,{'view': 1},4.05,4.050000,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1,117,1,4.05,1,0,0,0,1,0.0,0.0,0.00,1.00,15,31120.000000,4.050000,1.0,0
2,1180452,{'view': 1},25.40,25.400000,1,2019-12-28 14:32:56+00:00,2019-12-28 14:32:56+00:00,0,1,63,1,25.40,1,0,0,0,1,0.0,0.0,0.00,1.00,14,119609.000000,25.400000,1.0,1
3,1458813,{'view': 29},159.36,5.495172,4,2019-11-19 09:56:09+00:00,2019-11-21 06:02:40+00:00,0,5,100,4,159.36,29,0,0,0,29,0.0,0.0,947.25,7.25,12,71265.448276,5.495172,1.0,0
4,2038666,{'view': 1},25.38,25.380000,1,2020-02-16 13:19:37+00:00,2020-02-16 13:19:37+00:00,0,1,13,1,25.38,1,0,0,0,1,0.0,0.0,0.00,1.00,13,28437.000000,25.380000,1.0,1


In [74]:
user_agg['cluster'].value_counts()

cluster
1    803423
0    736185
2     98162
Name: count, dtype: int64

In [75]:
df_c = df.copy()

In [85]:
df_with_segments = pd.merge(df_c, user_agg[['user_id','cluster']], on = 'user_id',how= 'left')

# funnel analysis by kmeans cluster

funnel_by_cluster = df_with_segments.groupby(['cluster','event_type']).size().unstack().fillna(0)

funnel_by_cluster['view_to_cart'] = funnel_by_cluster['cart']/funnel_by_cluster['view']
funnel_by_cluster['cart_to_purchase'] = funnel_by_cluster['purchase']/funnel_by_cluster['cart']
funnel_by_cluster['dropoff_cart'] = 1- funnel_by_cluster['view_to_cart']
funnel_by_cluster['dropoff_purchase'] = 1- funnel_by_cluster['cart_to_purchase']
funnel_by_cluster['remove_from_cart_rate'] = funnel_by_cluster['remove_from_cart']/funnel_by_cluster['view']
funnel_by_cluster['view_to_remove'] = funnel_by_cluster['remove_from_cart']/funnel_by_cluster['view']

In [77]:
funnel_by_cluster

event_type,cart,purchase,remove_from_cart,view,view_to_cart,cart_to_purchase,dropoff_cart,dropoff_purchase,remove_from_cart_rate,view_to_remove
cluster,,,,,,,,,,
0,875405,33237,334744,1979472,0.442242,0.037968,0.557758,0.962032,0.169108,0.169108
1,1073359,47976,548668,2920602,0.367513,0.044697,0.632487,0.955303,0.187861,0.187861
2,3766424,1205667,3071177,4727224,0.796752,0.320109,0.203248,0.679891,0.649679,0.649679


In [86]:
funnel_by_cluster[['view_to_cart','cart_to_purchase','dropoff_cart','dropoff_purchase','remove_from_cart_rate','view_to_remove']] = \
                                            (funnel_by_cluster[['view_to_cart','cart_to_purchase','dropoff_cart','dropoff_purchase','remove_from_cart_rate','view_to_remove']]*100).round(2)

In [87]:
funnel_by_cluster

event_type,cart,purchase,remove_from_cart,view,view_to_cart,cart_to_purchase,dropoff_cart,dropoff_purchase,remove_from_cart_rate,view_to_remove
cluster,,,,,,,,,,
0,875405,33237,334744,1979472,44.22,3.80,55.78,96.20,16.91,16.91
1,1073359,47976,548668,2920602,36.75,4.47,63.25,95.53,18.79,18.79
2,3766424,1205667,3071177,4727224,79.68,32.01,20.32,67.99,64.97,64.97


In [144]:
cluster_summary = user_agg.groupby('cluster').agg({
    'recency' : 'mean',
    'frequency': 'mean',
    'monetary': 'mean',
    'conversion_rate': 'mean',
    'cart_abandonment_rate': 'mean',
    'avg_session_duration': 'mean',
    'category_diversity': 'mean',
    'avg_category_freq': 'mean',
    'price_ratio': 'mean',
    'user_id': 'count'
}). rename(columns = {'user_id': 'user_count'})

cluster_summary

,recency,frequency,monetary,conversion_rate,cart_abandonment_rate,avg_session_duration,category_diversity,avg_category_freq,price_ratio,user_count
cluster,,,,,,,,,,
0,113.583713,1.647762,50.980604,0.001693,15300.551716,332.586254,1.624422,219796.095329,230898.293836,736185
1,34.165107,2.133430,66.042904,0.001929,7539.055319,419.743074,1.928857,204485.477935,177053.345645,803423
2,52.729783,16.233471,875.943812,0.176851,38202.714496,4875.485415,16.528280,260945.913325,479965.564431,98162


In [123]:
user_agg_rfm = user_agg.copy()

In [124]:
user_agg_rfm['frequency'].nunique()

514

In [126]:
user_agg_rfm['r_quartile'] = pd.qcut(user_agg_rfm['recency'], 4, labels=[4,3,2,1])
user_agg_rfm['f_quartile'] = pd.qcut(user_agg_rfm['frequency'].rank(method = 'first'), 4, labels=[1,2,3,4])
user_agg_rfm['m_quartile'] = pd.qcut(user_agg_rfm['monetary'].clip(lower=0), 4, labels=[1,2,3,4])
user_agg_rfm['rfm_score'] = user_agg_rfm[['r_quartile','f_quartile','m_quartile']].sum(axis = 1)
# user_agg_rfm['rfm_score'] = user_agg_rfm[['r_quartile','m_quartile']].sum(axis = 1)

def rfm_segment(row):
  if row['rfm_score']>=10:
    return 'VIP'
  elif row['rfm_score'] >=7:
    return 'Active'
  elif row['rfm_score']>=4:
    return 'At-Risk'
  else:
    return 'Churned'

user_agg_rfm['rfm_segment'] = user_agg_rfm.apply(rfm_segment, axis=1)
user_agg_rfm.head()

,user_id,event_counts,total_spend,avg_price,session_count,first_event,last_event,purchase_count,category_diversity,recency,frequency,monetary,view_count,remove_count,cart_count,purchase_event_count,total_events,conversion_rate,cart_abandonment_rate,avg_session_duration,avg_events_per_session,most_common_hour,avg_category_freq,avg_view_price,price_ratio,cluster,r_quartile,f_quartile,m_quartile,rfm_score,rfm_segment
0,465496,{'view': 3},30.00,10.000000,2,2020-01-14 16:15:21+00:00,2020-01-14 16:27:31+00:00,0,1,46,2,30.00,3,0,0,0,3,0.0,0.0,58.00,1.50,16,39226.000000,10.000000,1.0,1,3,3,3,9,Active
1,1120748,{'view': 1},4.05,4.050000,1,2019-11-04 15:28:57+00:00,2019-11-04 15:28:57+00:00,0,1,117,1,4.05,1,0,0,0,1,0.0,0.0,0.00,1.00,15,31120.000000,4.050000,1.0,0,1,1,1,3,Churned
2,1180452,{'view': 1},25.40,25.400000,1,2019-12-28 14:32:56+00:00,2019-12-28 14:32:56+00:00,0,1,63,1,25.40,1,0,0,0,1,0.0,0.0,0.00,1.00,14,119609.000000,25.400000,1.0,1,3,1,3,7,Active
3,1458813,{'view': 29},159.36,5.495172,4,2019-11-19 09:56:09+00:00,2019-11-21 06:02:40+00:00,0,5,100,4,159.36,29,0,0,0,29,0.0,0.0,947.25,7.25,12,71265.448276,5.495172,1.0,0,2,4,4,10,VIP
4,2038666,{'view': 1},25.38,25.380000,1,2020-02-16 13:19:37+00:00,2020-02-16 13:19:37+00:00,0,1,13,1,25.38,1,0,0,0,1,0.0,0.0,0.00,1.00,13,28437.000000,25.380000,1.0,1,4,1,3,8,Active


In [127]:
user_agg_rfm = user_agg.copy()
df_rfm = df.copy()

In [128]:
df_rfm_with_segments = df_rfm.merge(user_agg_rfm[['user_id', 'rfm_segment']], on='user_id', how='left')

# Funnel analysis by RFM segments
rfm_funnel = df_rfm_with_segments.groupby(['rfm_segment', 'event_type']).size().unstack().fillna(0)
rfm_funnel['view_to_cart'] = rfm_funnel['cart'] / rfm_funnel['view']
rfm_funnel['cart_to_purchase'] = rfm_funnel['purchase'] / rfm_funnel['cart']
rfm_funnel['dropoff_cart'] = 1 - rfm_funnel['view_to_cart']
rfm_funnel['dropoff_purchase'] = 1 - rfm_funnel['cart_to_purchase']

rfm_funnel['remove_from_cart_rate'] = rfm_funnel['remove_from_cart']/rfm_funnel['view']
rfm_funnel['view_to_remove'] = rfm_funnel['remove_from_cart']/rfm_funnel['view']
print("Funnel Analysis by RFM Segments")
rfm_funnel

Funnel Analysis by RFM Segments


event_type,cart,purchase,remove_from_cart,view,view_to_cart,cart_to_purchase,dropoff_cart,dropoff_purchase,remove_from_cart_rate,view_to_remove
rfm_segment,,,,,,,,,,
Active,961812,202765,483464,1965667,0.489306,0.210816,0.510694,0.789184,0.245954,0.245954
At-Risk,347160,31632,62008,749868,0.462961,0.091116,0.537039,0.908884,0.082692,0.082692
Churned,18170,6,257,79986,0.227165,0.000330,0.772835,0.999670,0.003213,0.003213
VIP,4388046,1052477,3408860,6831777,0.642299,0.239851,0.357701,0.760149,0.498971,0.498971


In [129]:
rfm_funnel[['view_to_cart', 'cart_to_purchase', 'dropoff_cart', 'dropoff_purchase']] = (rfm_funnel[['view_to_cart', 'cart_to_purchase', 'dropoff_cart', 'dropoff_purchase']]* 100).round(2) 

In [131]:
# RFM segment summary
rfm_summary = user_agg_rfm.groupby('rfm_segment').agg({
    'recency': 'mean', 'frequency': 'mean', 'monetary': 'mean',
    'conversion_rate': 'mean', 'cart_abandonment_rate': 'mean',
    'avg_session_duration': 'mean', 'category_diversity': 'mean',
    'avg_category_freq': 'mean', 'price_ratio': 'mean',
    'user_id': 'count'
}).rename(columns={'user_id': 'user_count'}).round(3)
print("RFM Segment Summary")
rfm_summary

RFM Segment Summary


,recency,frequency,monetary,conversion_rate,cart_abandonment_rate,avg_session_duration,category_diversity,avg_category_freq,price_ratio,user_count
rfm_segment,,,,,,,,,,
Active,59.903,1.722,58.853,0.010,12347.056,691.305,1.872,207467.758,264860.356,626468
At-Risk,102.590,1.044,20.445,0.003,10949.415,333.630,1.196,214522.193,229783.645,540121
Churned,135.980,1.000,3.256,0.000,1371.921,80.918,1.012,232511.594,62325.122,77264
VIP,32.492,7.109,325.987,0.030,18572.688,1119.856,6.274,223166.984,163724.775,393917
